# 1. Librerías

In [1]:
import pandas as pd
import numpy as np
import os
import datetime
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.neighbors import KNeighborsRegressor

# 2. Importación y transformación de los datos

## 2.1 Ficheros Best players

In [ ]:
#Importamos los ficheros CSV y los agrupamos en un mismo df

directory = r'Datasets\Best Players'

dataframes = []

for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        df = pd.read_csv(os.path.join(directory, filename))
        dataframes.append(df)

best_players_raw = pd.concat(dataframes, ignore_index=True)

output_path = r'Datasets\Best Players\best_players_raw.csv'

best_players_raw.to_csv(output_path, index=False)


In [ ]:
#Eliminamos las columnas que no necesitamos para el análisis

best_players_raw = best_players_raw.drop(['Unnamed: 8','Unnamed: 15','Finish','SRS','Pace','Rel Pace','ORtg','Rel ORtg','DRtg','Rel DRtg','Coaches'], axis=1)

In [ ]:
#Eliminamos un signo que no es necesario

best_players_raw['Team'] = best_players_raw['Team'].str.replace('*', '', regex=False)

In [ ]:
#Dividimos la columna en questión para trabajar mejor con los datos

best_players_raw[['Player', 'WS']] = best_players_raw['Top WS'].str.split('(', expand=True)

best_players_raw['WS'] = best_players_raw['WS'].str.rstrip(')')

best_players_raw = best_players_raw.drop(['Top WS'], axis=1)


In [ ]:
#Creamos una nueva columna para trabajar mejor con los datos

best_players_raw.rename(columns={'Playoffs': 'Playoffs Clasification'}, inplace=True)

best_players_raw['Playoff'] = np.where(best_players_raw['Playoffs Clasification'].isna(), 'No', 'Yes')

In [ ]:
#Comprobamos valores nulos
null_counts = best_players_raw.isna().sum()

In [ ]:
null_counts

In [ ]:
#Guardamos el DataFrame como un archivo CSV

df = best_players_raw

output_path = "best_players.csv" 

df.to_csv(output_path, index=False)

## 2.2 Draft Picks

In [ ]:
#Importamos los ficheros CSV y los agrupamos en un mismo df

directory = r'Datasets\Draft Picks'

dataframes = []

for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        df = pd.read_csv(os.path.join(directory, filename))
        df['filename'] = filename[:-4]
        dataframes.append(df)


draft_picks_raw = pd.concat(dataframes, ignore_index=True)

output_path = r'Datasets\Draft Picks\draft_picks_raw.csv'

draft_picks_raw.to_csv(output_path, index=False)

In [ ]:
draft_picks_raw = pd.read_csv(r'Datasets\Draft Picks\draft_picks_raw.csv')

In [ ]:
#Eliminamos el .0 de Year
draft_picks_raw['Year'] = draft_picks_raw['Year'].astype(str).str.replace('.0', '', regex=False)

In [ ]:
#Eliminamos ligas anteriores a NBA
draft_picks_raw = draft_picks_raw.loc[~draft_picks_raw['Lg'].isin(['BAA'])]

In [ ]:
#Eliminamos las columnas que no necesitamos para el análisis
draft_picks_raw = draft_picks_raw.drop(['Lg','BPM','WS/48','VORP'], axis=1)

In [ ]:
#Limpiamos los nulos
draft_picks_raw['College'] = draft_picks_raw['College'].fillna('Out US')
draft_picks_raw = draft_picks_raw.fillna(0)

In [ ]:
#Limpiamos variable Rd
draft_picks_raw['Rd'] = draft_picks_raw['Rd'] .str.replace('[T]', '', regex=True).astype(float)

In [ ]:
# Eliminamos las filas de jugadores que no han disputado ningún partido
draft_picks_raw = draft_picks_raw[draft_picks_raw['G'] != 0]

In [ ]:
# Eliminamos las filas de jugadores que en 1952-1953 no fueron drafteados y están incluidos
draft_picks_raw = draft_picks_raw[draft_picks_raw['Pk'] != 0]

In [ ]:
#Convertimos algunos valores en enteros
draft_picks_raw[['Rd','Pk','G', 'MP', 'PTS', 'TRB', 'AST']] = draft_picks_raw[['Rd','Pk','G', 'MP', 'PTS', 'TRB', 'AST']].astype(int)

In [ ]:
#Cambiamos los nombres de algunas variables
draft_picks_raw.rename(columns={
                                'Pk': 'Pick',
                                'Rd': 'Round',
                                'filename':'Team', 
                                'G': 'Games',
                                'MP': 'Minutes Played',
                                'MP.1': 'Minutes Played per game',
                                'PTS.1': 'PTS per game',
                                'TRB.1': 'TBR per game',
                                'AST.1': 'AST per game',}, inplace=True)

In [ ]:
# Function to remove (↳GSW) from player names
def remove_team_initials(player_name):
    if isinstance(player_name, str):
        return player_name.split(' (↳')[0]
    else:
        return player_name  # Return as-is for non-string types

# Apply the function to the 'player' column
draft_picks_raw['Player'] = draft_picks_raw['Player'].apply(remove_team_initials)

In [ ]:
draft_picks_raw

In [ ]:
#Guardamos el DataFrame como un archivo CSV

df = draft_picks_raw

output_path = "draft_picks.csv" 

df.to_csv(output_path, index=False)

#Algunos equipos no aparecen en el dataset como Baltimore Bullets, de los 50

## 2.3 Payroll

In [ ]:
import csv

# Assuming your CSV file is named 'data.csv'
file_path = r'Datasets\Payroll\NBA Salary Cap History.csv'

# Specify the delimiter as '\t' (tab character)
delimiter = '\t'

# Manually define headers
headers = ['Year', 'Salary', 'Salary Adjusted to Year 2022']

# Initialize an empty list to store rows from the CSV
data = []

# Open the CSV file and read its contents using csv.reader
with open(file_path, 'r', newline='') as file:
    reader = csv.reader(file, delimiter=delimiter)
    
    # Skip the header row from the file
    next(reader)
    
    # Read remaining rows
    for row in reader:
        data.append(row)

# Create a pandas DataFrame
df = pd.DataFrame(data, columns=headers)

In [ ]:
#Convertimos en un valor numérico
df['Salary'] = df['Salary'].str.replace('[\$,]', '', regex=True).astype(float)
df['Salary Adjusted to Year 2022'] = df['Salary Adjusted to Year 2022'].str.replace('[\$,]', '', regex=True).astype(float)

In [ ]:
years_to_update = ['2022-23', '2023-24']


for year in years_to_update:
    mask = df['Year'] == year
    df.loc[mask, 'Salary Adjusted to Year 2022'] = df.loc[mask, 'Salary']

In [ ]:
#Transformamos la variable año para que se represente correctamente
def extract_year(year_str):
    return year_str.split('-')[0]

df['Year'] = df['Year'].apply(extract_year)
df['Year'] = df['Year'].astype(int)

In [ ]:
output_path = "NBA_Salary_History.csv" 

df.to_csv(output_path, index=False)

### 2.3 Payroll 2023-2024

In [2]:
df_payroll= pd.read_csv(r'Datasets\Payroll\2023-2024.csv')
df_players= pd.read_csv(r'Datasets Cleaned\player_stats.csv')

In [4]:
#Cambiamos los nombres de algunas variables
df_payroll.rename(columns={'2023-24': 'Salary','Tm': 'Team'}, inplace=True)

In [5]:
#Convertimos en un valor numérico
df_payroll['Salary'] = df_payroll['Salary'].str.replace('[\$,]', '', regex=True).astype(float)

In [6]:
df_2024 = df_players[df_players['Season'] == '2023-24']

In [7]:
#Acotamos las columnas de la base de datos  para centrarnos en las variables que se pueden aportar más información útil y agrupamos Datasets
target_columns = ['Player','Salary', 'Team']
df_2024 = pd.merge(df_2024, df_payroll[target_columns], on='Player', how='left')

In [10]:
#Hay valores nulos en Salary por lo que aplicamos KNN
df_2024 .isnull().sum()

Player     0
Pos        0
Age        0
G          0
MP         0
FG         0
FGA        0
FG%        0
3P         0
3PA        0
3P%        0
2P         0
2PA        0
2P%        0
FT         0
FTA        0
FT%        0
ORB        0
DRB        0
TRB        0
AST        0
STL        0
BLK        0
TOV        0
PF         0
PTS        0
OWS        0
DWS        0
WS         0
Season     0
Salary    96
Team      91
dtype: int64

In [11]:
def knn(df, columna_objetivo, min_k=2, max_k=15):
    """
    Dado un DataFrame de pandas, el nombre de una columna objetivo, un rango de valores k y un
    número mínimo de muestras por pliegue, realiza una regresión K-NN usando validación cruzada
    para encontrar el mejor valor de k (número de vecinos) basado en el error cuadrático medio.
    
    Parámetros:
    - df: DataFrame de pandas
    - columna_objetivo: str, nombre de la columna objetivo
    - min_k: int, número mínimo de vecinos a considerar
    - max_k: int, número máximo de vecinos a considerar
    
    Retorna:
    - mejor_k: int, mejor valor de k encontrado
    """
    #Instanciar un objeto LabelEncoder
    le = LabelEncoder()
    #Hacer una copia del DataFrame de entrada
    df_encoded = df.copy()
    #Seleccionar columnas de tipo objeto (categóricas) del df_encoded
    columnas_objeto = df_encoded.select_dtypes(include=["object"]).columns
    #Iterar sobre cada columna categórica y aplicar codificación de etiquetas
    for columna in columnas_objeto:
        df_encoded[columna] = le.fit_transform(df_encoded[columna].astype(str))
    #Imputar valores faltantes usando la media de cada columna
    imputador = SimpleImputer(strategy="mean")
    df_imputed = pd.DataFrame(
        imputador.fit_transform(df_encoded), columns=df_encoded.columns
    )
    #Separar los predictores (X) del objetivo (y)
    X = df_imputed.drop(columna_objetivo, axis=1)
    y = df_imputed[columna_objetivo]
    #Definir una tubería para la regresión K-NN
    pipeline = Pipeline(steps=[("modelo", KNeighborsRegressor(n_neighbors=3))])
    #Establecer los hiperparámetros a afinar
    parametros = {
        "modelo__n_neighbors": [3, 5, 7],
        "modelo__weights": ["uniform", "distance"],
    }
    mejor_k = 0
    mejor_puntaje = -np.inf
    #Iterar sobre un rango de valores k y realizar validación cruzada
    for k in range(min_k, max_k + 1):
        kf = KFold(n_splits=k, shuffle=True, random_state=42)
        grid_search = GridSearchCV(
            pipeline, parametros, cv=kf, scoring="neg_mean_squared_error", n_jobs=-1
        )
        grid_search.fit(X, y)
        #Mantener el mejor k y el mejor puntaje encontrados hasta ahora
        if grid_search.best_score_ > mejor_puntaje:
            mejor_puntaje = grid_search.best_score_
            mejor_k = k
    #Retornar el mejor valor de k encontrado
    return mejor_k

def imputar_valores_faltantes_con_knn(df, columna_objetivo, n_vecinos):
    """
    Imputa valores faltantes en la columna objetivo especificada usando imputación KNN.
    
    Parámetros:
    - df: DataFrame de pandas
    - columna_objetivo: str, nombre de la columna objetivo
    - n_vecinos: int, número de vecinos a usar para la imputación KNN
    
    Retorna:
    - df_imputed: DataFrame de pandas con valores imputados
    """
    df_imputed = df.copy()
    imputador = KNNImputer(n_neighbors=n_vecinos)
    columnas_numericas = df_imputed.select_dtypes(include=["int64", "float64"]).columns
    df_imputed[columnas_numericas] = imputador.fit_transform(df_imputed[columnas_numericas])
    return df_imputed

def tipo_de_datos_original(df_encoded, info_codificador):
    """
    Dado un DataFrame de pandas que ha sido codificado con la función `knn` y el diccionario de información del codificador
    devuelto por esa función, decodifica todas las columnas categóricas y retorna
    una copia del DataFrame original con las columnas codificadas reemplazadas por sus valores originales.
    Esta función reemplaza cualquier código que no esté en la lista original de etiquetas con -1.
    
    Parámetros:
    - df_encoded: DataFrame de pandas
    - info_codificador: lista de diccionarios
    
    Retorna:
    - df_decoded: DataFrame de pandas
    """
    df_decoded = df_encoded.copy()  # Hacer una copia del DataFrame codificado

    #Iterar sobre cada codificador en el diccionario info_codificador
    for codificador in info_codificador:
        columna = codificador["columna"]
        etiquetas = codificador["etiquetas"]
        le = LabelEncoder()  #Crear un nuevo LabelEncoder para la columna
        le.classes_ = np.array(etiquetas)  #Establecer las etiquetas originales
        
        #Detectar -1 y convertir a NaN para la transformación inversa
        mascara = df_decoded[columna] == -1
        df_decoded[columna] = df_decoded[columna].where(~mascara, other=-1)
        
        #Transformar inversamente los valores no NaN
        mascara_no_nan = df_decoded[columna] != -1
        df_decoded.loc[mascara_no_nan, columna] = le.inverse_transform(df_decoded.loc[mascara_no_nan, columna].astype(int))
        
        #Convertir -1 de nuevo a NaN
        df_decoded[columna] = df_decoded[columna].replace(-1, np.nan)

    return df_decoded

# Cargar tu conjunto de datos
df_2024 

# Encontrar el mejor valor de k para la imputación
mejor_k = knn(df_2024, 'Salary', min_k=2, max_k=15)
print(f'Mejor valor de k: {mejor_k}')

# Imputar valores faltantes en la columna 'Salary' usando imputación KNN
df_2024_salary = imputar_valores_faltantes_con_knn(df_2024, 'Salary', mejor_k)

Mejor valor de k: 4


In [14]:
df_2024_salary.isnull().sum()

Player     0
Pos        0
Age        0
G          0
MP         0
FG         0
FGA        0
FG%        0
3P         0
3PA        0
3P%        0
2P         0
2PA        0
2P%        0
FT         0
FTA        0
FT%        0
ORB        0
DRB        0
TRB        0
AST        0
STL        0
BLK        0
TOV        0
PF         0
PTS        0
OWS        0
DWS        0
WS         0
Season     0
Salary     0
Team      91
dtype: int64

In [15]:
df_2024_salary

,Player,Pos,Age,G,MP,FG,FGA,FG%,3P,3PA,...,BLK,TOV,PF,PTS,OWS,DWS,WS,Season,Salary,Team
0,Precious Achiuwa,PF,24.0,74.0,1624.0,235.0,469.0,0.501,26.0,97.0,...,68.0,83.0,143.0,565.0,1.2,2.2,3.4,2023-24,4379527.0,NYK
1,Bam Adebayo,C,26.0,71.0,2416.0,530.0,1017.0,0.521,15.0,42.0,...,66.0,162.0,159.0,1367.0,2.9,4.3,7.2,2023-24,32600060.0,MIA
2,Ochai Agbaji,SG,23.0,78.0,1641.0,178.0,433.0,0.411,62.0,211.0,...,44.0,64.0,117.0,455.0,-0.5,0.6,0.1,2023-24,4114200.0,TOR
3,Santi Aldama,PF,23.0,61.0,1618.0,247.0,568.0,0.435,106.0,304.0,...,54.0,69.0,89.0,654.0,0.4,2.0,2.4,2023-24,2194200.0,MEM
4,Nickeil Alexander-Walker,SG,25.0,82.0,1921.0,236.0,538.0,0.439,131.0,335.0,...,42.0,76.0,143.0,655.0,1.5,2.8,4.3,2023-24,4687500.0,MIN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617,Thaddeus Young,PF,35.0,33.0,439.0,65.0,108.0,0.602,1.0,7.0,...,5.0,15.0,49.0,137.0,0.9,0.4,1.3,2023-24,8638413.0,PHO
618,Trae Young,PG,25.0,54.0,1942.0,433.0,1008.0,0.430,175.0,469.0,...,11.0,235.0,109.0,1389.0,4.0,0.6,4.6,2023-24,40064220.0,ATL
619,Omer Yurtseven,C,25.0,48.0,545.0,99.0,184.0,0.538,5.0,24.0,...,18.0,37.0,52.0,222.0,0.3,0.4,0.7,2023-24,2800000.0,UTA
620,Cody Zeller,C,31.0,43.0,320.0,26.0,62.0,0.419,1.0,3.0,...,5.0,16.0,45.0,76.0,0.4,0.4,0.8,2023-24,2019706.0,NOP


In [16]:
#Hay valores nulos en Team, decidimos eliminarlos ya que juegan en la G Leage con contratos duales, o casos como Jontay Porter vetado de la liga profesional
df_2024_salary = df_2024_salary.dropna(subset=['Team'])

In [18]:
output_path = "salary_2024.csv" 

df_2024_salary.to_csv(output_path, index=False)

## 2.4 Team Stats

In [ ]:
#Importamos los ficheros CSV y los agrupamos en un mismo df

directory = r'Datasets\Team Stats'

dataframes = []

for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        df = pd.read_csv(os.path.join(directory, filename))
        df['filename'] = filename[:-4]
        dataframes.append(df)


Team_stats_raw = pd.concat(dataframes, ignore_index=True)

output_path = r'Datasets\Team Stats\Team_stats_raw.csv'

Team_stats_raw.to_csv(output_path, index=False)

In [ ]:
#Eliminamos ligas anteriores a NBA
Team_stats_raw = Team_stats_raw.loc[~Team_stats_raw['Lg'].isin(['BAA'])]

In [ ]:
#Eliminamos las columnas que no necesitamos para el análisis
Team_stats_raw = Team_stats_raw.drop(['Unnamed: 6','Unnamed: 10','G'], axis=1)

In [ ]:
#Cambiamos los nombres de algunas variables
Team_stats_raw.rename(columns={'Tm': 'Original Team Name', 
                                'filename': 'Current Team Name',
                                'Finish': 'Position Regular Season',
                                'Age': 'Average Player Age',
                                'Ht.': 'Average Player Height',
                                'Wt.': 'Average Player Weight',
                                'G': 'TBR per game'}, inplace=True)

In [ ]:
#Extraermos el año inicial de la temporada y los convertimos a entero, luego eliminamos filas donde el año es menor a 1979
df['Season'] = df['Season'].str[:4].astype(int)

df = df[df['Season'] >= 1979]

In [ ]:
#Guardamos el DataFrame como un archivo CSV

df = Team_stats_raw 

output_path = "team_stats.csv" 

df.to_csv(output_path, index=False)

## 2.5 Arena

In [ ]:
df = pd.read_csv(r'Datasets\Team Advanced Stats.csv')

In [ ]:
#Nos quedamos con la información de los estadios
columnas_deseadas = ['Arena', 'Attend.', 'Attend./G']
df = df[columnas_deseadas]

In [ ]:
#Eliminamos la fila total
df = df.dropna(subset=['Arena'])

In [ ]:
#Guardamos el DataFrame como un archivo CSV

output_path = "arena.csv" 

df.to_csv(output_path, index=False)

## 2.5 Player Stats

In [ ]:
df_2024 = pd.read_csv(r'Datasets\Player Stats\2023-24 NBA Player Stats.csv')
df_2024_adv = pd.read_csv(r'Datasets\Player Stats\2023-24 NBA Player Stats ADV.csv')

In [ ]:
#Eliminamos las columnas que no necesitamos para el análisis
df_2024 = df_2024.drop(['Rk','Tm','GS','eFG%'], axis=1)

In [ ]:
#Eliminamos los nulos
df_2024.loc[df_2024['3P'] == 0, '3P%'] = 0
df_2024.loc[df_2024['FT'] == 0, 'FT%'] = 0
df_2024.loc[df_2024['2P'] == 0, '2P%'] = 0
df_2024.loc[df_2024['FG'] == 0, 'FG%'] = 0

In [ ]:
#Acotamos las columnas de la base de datos  para centrarnos en las variables que se pueden aportar más información útil y agrupamos Datasets
target_columns = ["Player-additional", "OWS", "DWS", "WS"]
df_2024 = pd.merge(df_2024, df_2024_adv[target_columns], on='Player-additional', how='left')

In [ ]:
#Guardamos el DataFrame como un archivo CSV
output_path = "2023-24.csv" 
df_2024.to_csv(output_path, index=False)

In [ ]:
#Importamos los ficheros CSV generados y los agrupamos en un mismo df

directory = r'Datasets\Player Stats\Players'

dataframes = []

for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        df = pd.read_csv(os.path.join(directory, filename))
        df['filename'] = filename[:-4]
        dataframes.append(df)


players_stats_raw = pd.concat(dataframes, ignore_index=True)

output_path = 'players_stats_raw'

players_stats_raw.to_csv(output_path, index=False)

In [ ]:
players_stats_raw = pd.read_csv(r'Datasets\Player Stats\players_stats_raw')

In [ ]:
#Eliminamos las columnas que no necesitamos para el análisis
players_stats_raw  = players_stats_raw.drop(['Player-additional'], axis=1)

In [ ]:
#Cambiamos los nombres de algunas variables
players_stats_raw.rename(columns={'filename': 'Season'}, inplace=True)

In [ ]:
players_stats_raw['Pos'] = players_stats_raw['Pos'].replace('SG-PG', 'SG')
players_stats_raw['Pos'] = players_stats_raw['Pos'].replace('PF-SF', 'PF')
players_stats_raw['Pos'] = players_stats_raw['Pos'].replace('SG-SF', 'SG')
players_stats_raw['Pos'] = players_stats_raw['Pos'].replace('SF-PF', 'SF')
players_stats_raw['Pos'] = players_stats_raw['Pos'].replace('PG-SG', 'PG')
players_stats_raw['Pos'] = players_stats_raw['Pos'].replace('SF-SG', 'SF')
players_stats_raw['Pos'] = players_stats_raw['Pos'].replace('PF-C', 'PF')
players_stats_raw['Pos'] = players_stats_raw['Pos'].replace('SG-PF', 'SG')
players_stats_raw['Pos'] = players_stats_raw['Pos'].replace('C-PF', 'C')
players_stats_raw['Pos'] = players_stats_raw['Pos'].replace('SF-C', 'SF')
players_stats_raw['Pos'] = players_stats_raw['Pos'].replace('SG-PG-SF', 'SG')

In [ ]:
#Guardamos el DataFrame como un archivo CSV
output_path = "player_stats.csv" 
players_stats_raw.to_csv(output_path, index=False)